# Fundamentals of neural network: MCP and perceptron

In [2]:
def step_function(value, threshold):
  return 1 if value >= threshold else 0


step_function(2, 5)
step_function(7, 5)

0

1

## MCP neuron

In [3]:
class MCPNeuron:
  def __init__(self, threshold, inhibitory_indices=None):
    self.threshold = threshold
    self.inhibitory_indices = inhibitory_indices if inhibitory_indices else []

  def run(self, inputs):
    for i in self.inhibitory_indices:
      if inputs[i] == 1:
        return 0

    inputs_sum = sum(inputs)
    return step_function(inputs_sum, self.threshold)


mcp_neuron = MCPNeuron(threshold=2)
print(mcp_neuron.run([0, 0, 0]))
print(mcp_neuron.run([0, 0, 1]))
print(mcp_neuron.run([1, 0, 1]))

0
0
1


In [4]:
mcp_neuron = MCPNeuron(threshold=3)
print(mcp_neuron.run([0, 0, 0]))
print(mcp_neuron.run([0, 0, 1]))
print(mcp_neuron.run([1, 0, 1]))
print(mcp_neuron.run([1, 1, 1]))

0
0
0
1


In [5]:
mcp_neuron = MCPNeuron(threshold=2, inhibitory_indices=[0, 1])
print(mcp_neuron.run([0, 0, 0, 1, 0]))
print(mcp_neuron.run([0, 0, 1, 1, 0]))
print(mcp_neuron.run([1, 0, 1, 1, 0]))
print(mcp_neuron.run([1, 1, 1, 1, 0]))

0
1
0
0


## MCP logic gates

In [6]:
mcp_AND = MCPNeuron(threshold=2)

print(mcp_AND.run([0, 0]))  # 0 + 0 => 0
print(mcp_AND.run([1, 0]))  # 1 + 0 => 1
print(mcp_AND.run([0, 1]))
print(mcp_AND.run([1, 1]))  # 1 + 1 => 2

0
0
0
1


In [7]:
mcp_OR = MCPNeuron(threshold=1)

print(mcp_OR.run([0, 0]))
print(mcp_OR.run([1, 0]))  # 1 + 0 => 1
print(mcp_OR.run([0, 1]))
print(mcp_OR.run([1, 1]))

0
1
1
1


In [8]:
mcp_NOT = MCPNeuron(threshold=0, inhibitory_indices=[0])

print(mcp_NOT.run([0]))
print(mcp_NOT.run([1]))

1
0


In [9]:
# NAND
print(mcp_NOT.run([mcp_AND.run([0, 0])]))
print(mcp_NOT.run([mcp_AND.run([1, 0])]))
print(mcp_NOT.run([mcp_AND.run([0, 1])]))
print(mcp_NOT.run([mcp_AND.run([1, 1])]))

1
1
1
0


In [10]:
def mcp_NAND(inputs):
  and_result = mcp_AND.run(inputs)
  return mcp_NOT.run([and_result])


print(mcp_NAND([0, 0]))
print(mcp_NAND([1, 0]))
print(mcp_NAND([0, 1]))
print(mcp_NAND([1, 1]))

1
1
1
0


In [11]:
mcp_NOR = MCPNeuron(threshold=0, inhibitory_indices=[0, 1])

print(mcp_NOR.run([0, 0]))
print(mcp_NOR.run([1, 0]))
print(mcp_NOR.run([0, 1]))
print(mcp_NOR.run([1, 1]))

1
0
0
0


## MCP TOY SCENARIO: Should I go to the beach?

In [12]:
#  Inputs (all binary):
#    inputs[0] : is_sunny       (excitatory)
#    inputs[1] : is_weekend     (excitatory)
#    inputs[2] : is_hot         (excitatory)
#    inputs[3] : shark_warning  (inhibitory) ← suppresses everything
#    inputs[4] : car_is_broken  (inhibitory) ← suppresses everything
#
#  Logic: go to beach if at least 2 of the 3 good conditions are met
#         AND neither inhibitory input is active
#
#  threshold = 2  (need at least 2 out of 3 excitatory inputs)
#  inhibitory_indices = [3, 4]

In [13]:
beach_neuron = MCPNeuron(threshold=2, inhibitory_indices=[3, 4])

print(beach_neuron.run([0, 0, 0, 0, 0]))  # -> 0
print(beach_neuron.run([1, 0, 0, 0, 0]))  # sunny -> 0
print(beach_neuron.run([1, 1, 0, 0, 0]))  # sunny, weekend -> 1
print(beach_neuron.run([1, 1, 1, 0, 0]))  # sunny, weekend, hot -> 1
print(beach_neuron.run([1, 1, 1, 1, 0]))  # sunny, weekend, hot, shark -> 0
print(beach_neuron.run([1, 0, 1, 1, 0]))  # sunny, hot, shark -> 0
print(beach_neuron.run([1, 0, 1, 0, 1]))  # sunny, hot, car broken -> 0
print(beach_neuron.run([1, 1, 1, 1, 1]))  # all -> 0

0
0
1
1
0
0
0
0


## PERCEPTRON

In [ ]:
class Perceptron:
  def __init__(self, weights, bias, threshold=0):
    self.weights = weights
    self.bias = bias
    self.threshold = threshold

  def run(self, inputs):
    weighted_sum = sum(w * x for w, x in zip(self.weights, inputs))
    total = weighted_sum + self.bias
    return step_function(total, self.threshold)


perceptron = Perceptron(weights=[1, 1, 1], bias=-1)
print(perceptron.run([1, 1, 1]))  # 1 + 1 + 1 + (-1) = 2 => threshold
print(perceptron.run([1, 1, 0]))  # 1 + 1 + 0 + (-1) = 1 => threshold
print(perceptron.run([0, 0, 0]))

1
1
0


In [21]:
# bias [-2, -1)
perc_AND = Perceptron(weights=[1, 1], bias=-2)

print(perc_AND.run([0, 0]))  # 0 + 0 -2 = -2
print(perc_AND.run([1, 0]))
print(perc_AND.run([0, 1]))
print(perc_AND.run([1, 1]))  # 1 + 1 -2 = 0

0
0
0
1


In [16]:
# bias [-1, 0)
perc_OR = Perceptron(weights=[1, 1], bias=-1)

print(perc_OR.run([0, 0]))
print(perc_OR.run([1, 0]))
print(perc_OR.run([0, 1]))
print(perc_OR.run([1, 1]))

0
1
1
1


In [22]:
# bias [0, 1)
perc_NOT = Perceptron(weights=[-1], bias=0)

print(perc_NOT.run([0]))  # -0 => 0
print(perc_NOT.run([1]))  # -1 + 0 => -1

1
0


##  PERCEPTRON TOY SCENARIO: Should I buy this laptop?

In [18]:
#  Inputs (real-valued, NOT binary):
#    inputs[0] : ram_gb           (e.g. 16)
#    inputs[1] : battery_hours    (e.g. 8.5)
#    inputs[2] : price_lakhs      (e.g. 1.2)  ← high price is bad
#
#  Weights reflect importance and direction:
#    ram         → positive weight (more RAM = better)
#    battery     → positive weight (more battery = better)
#    price       → negative weight (higher price = worse)
#
#  Bias adjusts the baseline.
#
#  Tune: good laptop = (0.5*ram) + (1.0*battery) + (-2.0*price) + bias >= 0

In [25]:
laptop_perceptron = Perceptron(weights=[0.5, 1.0, -15.0], bias=2.0)

print(laptop_perceptron.run([2, 10, 1]))  # 2 * 0.5 + 10 * 1 + 1 * -15 + 2 = -2
print(laptop_perceptron.run([4, 10, 0.5]))  # 4 * 0.5 + 10 * 1 + 0.5 * -15 + 2 = 6.5
print(laptop_perceptron.run([16, 10, 2]))
print(laptop_perceptron.run([16, 10, 3]))

0
1
0
0


In [20]:
laptop_scenarios = [
    [16, 10, 0.8],
    [8,  5,  1.5],
    [32, 12, 2.5],
    [4,  3,  0.3],
    [16, 8,  1.2],
]

for inputs in laptop_scenarios:
  result = laptop_perceptron.run(inputs)
  decision = "BUY IT ✅" if result == 1 else "SKIP IT ❌"
  weighted_sum = sum(w * x for w, x in zip(laptop_perceptron.weights, inputs))
  total = weighted_sum + laptop_perceptron.bias
  print(f'{inputs[0]}GB RAM, {inputs[1]}hr battery, Rs. {inputs[2]} lacs')
  print(f"  {weighted_sum=:.2f}  →  {total=:.2f} →  {decision}\n")

16GB RAM, 10hr battery, Rs. 0.8 lacs
  weighted_sum=6.00  →  total=8.00 →  BUY IT ✅

8GB RAM, 5hr battery, Rs. 1.5 lacs
  weighted_sum=-13.50  →  total=-11.50 →  SKIP IT ❌

32GB RAM, 12hr battery, Rs. 2.5 lacs
  weighted_sum=-9.50  →  total=-7.50 →  SKIP IT ❌

4GB RAM, 3hr battery, Rs. 0.3 lacs
  weighted_sum=0.50  →  total=2.50 →  BUY IT ✅

16GB RAM, 8hr battery, Rs. 1.2 lacs
  weighted_sum=-2.00  →  total=0.00 →  BUY IT ✅

